# Tutorial

In this tutorial, we will generate a jellium model Hamiltonian, prepare a mean-field state of this model, and perform time evolution via a Trotter-Suzuki product formula using two different methods.

## Model generation

We begin by generating a jellium model in the "plane wave dual basis".

In [1]:
import openfermion

# Set parameters of jellium model
dimensions = 2
grid_length = 2
scale = 1.0
spinless = True

# Construct the model
grid = openfermion.Grid(dimensions, grid_length, scale)
model = openfermion.jellium_model(
    grid, spinless=spinless, plane_wave=False)

print(model)

19.739208802178716 [0^ 0] +
-9.869604401089358 [0^ 2] +
-0.07957747154594767 [0^ 0 2^ 2] +
-9.869604401089358 [0^ 1] +
-0.07957747154594766 [0^ 0 1^ 1] +
-0.238732414637843 [0^ 0 3^ 3] +
-9.869604401089358 [2^ 0] +
-0.07957747154594767 [2^ 2 0^ 0] +
19.739208802178716 [2^ 2] +
-0.238732414637843 [2^ 2 1^ 1] +
-9.869604401089358 [2^ 3] +
-0.07957747154594766 [2^ 2 3^ 3] +
-9.869604401089358 [1^ 0] +
-0.07957747154594766 [1^ 1 0^ 0] +
-0.238732414637843 [1^ 1 2^ 2] +
19.739208802178716 [1^ 1] +
-9.869604401089358 [1^ 3] +
-0.07957747154594767 [1^ 1 3^ 3] +
-0.238732414637843 [3^ 3 0^ 0] +
-9.869604401089358 [3^ 2] +
-0.07957747154594766 [3^ 3 2^ 2] +
-9.869604401089358 [3^ 1] +
-0.07957747154594767 [3^ 3 1^ 1] +
19.739208802178716 [3^ 3]


As is evident from the printed output, the jellium model has the form

$$\sum_{p, q} T_{pq} a^\dagger_p a_q + \sum_{p, q} V_{pq} a^\dagger_p a_p a^\dagger_q a_q$$

in the plane wave dual basis. This expression contains $O(N^2)$ terms, in contrast to the $O(N^4)$ terms of the general electronic structure Hamiltonian. The Hamiltonian simulation algorithms in OpenFermion-Cirq are implemented specifically for Hamiltonians of this form. They take as input the OpenFermion data structure DiagonalCoulombHamiltonian, which represents such a Hamiltonian by storing the matrices $T_{pq}$, which describes one-body interactions, and $V_{pq}$, which describes two-body interactions.

Let's convert our model to an instance of DiagonalCoulombHamiltonian.

In [2]:
hamiltonian = openfermion.get_diagonal_coulomb_hamiltonian(model)

print(hamiltonian.one_body)
print()
print(hamiltonian.two_body)

[[19.7392088+0.j -9.8696044+0.j -9.8696044+0.j  0.       +0.j]
 [-9.8696044+0.j 19.7392088+0.j  0.       +0.j -9.8696044+0.j]
 [-9.8696044+0.j  0.       +0.j 19.7392088+0.j -9.8696044+0.j]
 [ 0.       +0.j -9.8696044+0.j -9.8696044+0.j 19.7392088+0.j]]

[[ 0.         -0.07957747 -0.07957747 -0.23873241]
 [-0.07957747  0.         -0.23873241 -0.07957747]
 [-0.07957747 -0.23873241  0.         -0.07957747]
 [-0.23873241 -0.07957747 -0.07957747  0.        ]]


## State preparation

To begin our simulation we will prepare a mean-field state, which is an eigenstate of the one-body term $\sum_{p, q} T_{pq} a^\dagger_p a_q$. This term is a quadratic Hamiltonian, so its eigenstates can be prepared by applying a Bogoliubov transformation to a computational basis (Fock) state. The Bogoliubov transformation changes the basis to one in which the quadratic Hamiltonian has the diagonal form $\sum_{p} \varepsilon_p b^\dagger_p b_p$, where the $b^\dagger_p$ are the creation operators for a new set of orbitals. We'll set the number of electrons to be half the total number of orbitals.

We'll use the OpenFermion class QuadraticHamiltonian to obtain the Bogoliubov transformation matrix. Then, we'll initialize some qubits and create a circuit that applies the transformation to these qubits. Since our algorithms work on a computer architecture with linear qubit connectivity, we'll use the `LineQubit` class.

In [3]:
# Initialize a computational basis state at half filling
filling_fraction = 0.5
n_modes = grid_length**dimensions * (2 - spinless)
n_electrons = int(filling_fraction * n_modes)
initial_state = int('1' * n_electrons + '0' * (n_modes - n_electrons),
                    base=2)

# Obtain the Bogoliubov transformation matrix
quad_ham = openfermion.QuadraticHamiltonian(hamiltonian.one_body)
transformation_matrix = quad_ham.diagonalizing_bogoliubov_transform()

# Create a circuit that prepares the mean-field state
import cirq
import openfermioncirq

qubits = cirq.LineQubit.range(n_modes)
state_preparation_circuit = cirq.Circuit.from_ops(
    openfermioncirq.bogoliubov_transform(
        qubits, transformation_matrix, initial_state=initial_state)
)

print(state_preparation_circuit)

0: ────────────────YXXY^-1.0────────────────────────────────────
                   │
1: ───YXXY^0.667───#2──────────Z^0.0─────────YXXY^0.5───────────
      │                                      │
2: ───#2───────────Z^0.0───────YXXY^-0.392───#2─────────Z^0.0───
                               │
3: ────────────────────────────#2────────────Z^0.0──────────────


Let's get rid of those pointless 0-degree Z-rotations by performing an optimization pass.

In [4]:
def optimize_circuit(circuit):
    cirq.DropNegligible().optimize_circuit(circuit)

optimize_circuit(state_preparation_circuit)

print(state_preparation_circuit)

0: ────────────────YXXY^-1.0────────────────────────────
                   │
1: ───YXXY^0.667───#2────────────────────────YXXY^0.5───
      │                                      │
2: ───#2───────────────────────YXXY^-0.392───#2─────────
                               │
3: ────────────────────────────#2───────────────────────


## Hamiltonian simulation via a Trotter-Suzuki product formula

The goal of Hamiltonian time evolution simulation is to apply the unitary operator $\exp(-i H t)$ for some time t. A simulation via a product formula proceeds by dividing the total evolution time $t$ into a finite number of steps $r$ and performing an approximate simulation of $\exp(-i H t/r)$ $r$ times. Each simulation of $\exp(-i H t/r)$ is called a Trotter step. The unitary $\exp(-i H t/r)$ is approximated by interleaving simulations of the terms $H_j$ of a decomposition $H = \sum_{j=1}^L H_j$. For example, the first-order symmetric, commonly known as the second-order, Trotter formula is

$$\exp(-i H t) \approx \prod_{j=1}^L \exp(-i H_j t/2) \prod_{j=L}^1 \exp(-i H_j t/2).$$

Higher-order product formulas are obtained from this one via a recursive construction. In our case, the $H_j$ have the form $T_{pq} a^\dagger_p a_q + T_{pq}^* a^\dagger_q a_p$ or $V_{pq} a^\dagger_p a_p a^\dagger_q a_q$.

To construct a circuit for performing time evolution via a product formula, we need to specify the total evolution time, the number of steps to use, and the order of the formula to use. For a fixed evolution time, increasing the number of steps and increasing the order of the formula both yield a more accurate simulation at the cost of increasing the gate count of the circuit.

We have the additional option of choosing between two different methods of implementing the second-order Trotter step, the base case of the recursive construction of higher-order formulas. The options are `SWAP_NETWORK` and `SPLIT_OPERATOR`, and they correspond to different orderings of the terms $H_j$ in the product formula. Different orderings give different results because the $H_j$ do not all commute. Both algorithms require only a linear connectivity architecture and perform a second-order Trotter step in linear depth, but `SWAP_NETWORK` results in slightly smaller depth. The default method is `SWAP_NETWORK`.

Let's construct a circuit with the `SWAP_NETWORK` (default) method using just one Trotter step and the second-order formula. We'll insert operations into the circuit using the strategy `EARLIEST` so the printed output will be most compact. Still, the circuit will be longer than the width of this notebook, so we'll print it out transposed.

In [5]:
# Set algorithm parameters
time = 1.
n_steps = 1
order = 1  # Use the first-order symmetric (second-order) formula

# Construct circuit
swap_network_trotter_step = cirq.Circuit.from_ops(
    openfermioncirq.simulate_trotter(
        qubits, hamiltonian, time, n_steps, order),
    strategy=cirq.InsertStrategy.EARLIEST)

optimize_circuit(swap_network_trotter_step)

print(swap_network_trotter_step.to_text_diagram(transpose=True))

0          1        2          3
│          │        │          │
XXYY^0.858─XXYY     XXYY^0.858─XXYY
│          │        │          │
@^0.0253───Z        @^0.0253───Z
│          │        │          │
×ᶠ─────────×ᶠ       ×ᶠ─────────×ᶠ
│          │        │          │
│          @^0.076──Z          │
│          │        │          │
│          ×ᶠ───────×ᶠ         │
│          │        │          │
XXYY^0.858─XXYY     XXYY^0.858─XXYY
│          │        │          │
@^0.0253───Z        @^0.0253───Z
│          │        │          │
×ᶠ─────────×ᶠ       ×ᶠ─────────×ᶠ
│          │        │          │
Z^-0.283   │        │          Z^-0.283
│          │        │          │
│          @^0.076──Z          │
│          │        │          │
│          ×ᶠ───────×ᶠ         │
│          │        │          │
│          Z^-0.283 Z^-0.283   │
│          │        │          │
│          Z^0.076──@          │
│          │        │          │
│          ×ᶠ───────×ᶠ         │
│          │        │       

Now let's do the same, but using the `SPLIT_OPERATOR` method.

In [6]:
split_operator_trotter_step = cirq.Circuit.from_ops(
    openfermioncirq.simulate_trotter(
        qubits, hamiltonian, time, n_steps, order,
        algorithm=openfermioncirq.trotter.SPLIT_OPERATOR),
    strategy=cirq.InsertStrategy.EARLIEST)

optimize_circuit(split_operator_trotter_step)

print(split_operator_trotter_step.to_text_diagram(transpose=True))

0           1           2           3
│           │           │           │
│           │           YXXY^0.5────#2
│           │           │           │
│           YXXY^0.608──#2          │
│           │           │           │
│           │           YXXY^-0.333─#2
│           │           │           │
YXXY^0.667──#2          │           │
│           │           │           │
│           YXXY────────#2          │
│           │           │           │
│           Z^0.858     YXXY^-0.392─#2
│           │           │           │
│           │           Z^0.858     Z^-0.283
│           │           │           │
│           │           YXXY^0.392──#2
│           │           │           │
│           YXXY^-1.0───#2          │
│           │           │           │
YXXY^-0.667─#2          │           │
│           │           │           │
│           │           YXXY^0.333──#2
│           │           │           │
│           YXXY^-0.608─#2          │
│           │           │           │


Let's run these circuits on the simulator that comes with Cirq and compute the energy of the resulting states. To guarantee that the simulator uses the same tensor product ordering as OpenFermion to interpret the initial state and construct the output wavefunction, we need to give it the qubit order explicitly.

In [7]:
simulator = cirq.google.XmonSimulator()

# Convert the Hamiltonian to a sparse matrix
hamiltonian_sparse = openfermion.get_sparse_operator(hamiltonian)

# Construct and simulate circuit using the swap network method
circuit = state_preparation_circuit + swap_network_trotter_step
result = simulator.simulate(
    circuit,
    initial_state=initial_state,
    qubit_order=qubits)
final_state = result.final_state

print('Energy of state obtained with swap network method: {}'.format(
    openfermion.expectation(hamiltonian_sparse, final_state).real))

# Construct and simulate circuit using the split-operator method
circuit = state_preparation_circuit + split_operator_trotter_step
result = simulator.simulate(
    circuit,
    initial_state=initial_state,
    qubit_order=qubits)
final_state = result.final_state

print('Energy of state obtained with split-operator method: {}'.format(
    openfermion.expectation(hamiltonian_sparse, final_state).real))

Energy of state obtained with swap network method: 24.719991853831573
Energy of state obtained with split-operator method: 20.373788628511623


Increasing the number of Trotter steps will cause both methods to converge to the same operation, corresponding to an exact simulation. Let's perform the simulation again after increasing the number of steps. This time, we'll construct both circuits from scratch.

In [8]:
# Increase the number of steps to 10
n_steps = 10

# Construct and simulate circuit using the swap network method
circuit = cirq.Circuit.from_ops(
    openfermioncirq.bogoliubov_transform(
        qubits, transformation_matrix, initial_state=initial_state),
    openfermioncirq.simulate_trotter(
        qubits, hamiltonian, time, n_steps, order,
        algorithm=openfermioncirq.trotter.SWAP_NETWORK)
)
result = simulator.simulate(
    circuit,
    initial_state=initial_state,
    qubit_order=qubits)
final_state = result.final_state

print('Energy of state obtained with swap network method: {}'.format(
    openfermion.expectation(hamiltonian_sparse, final_state).real))

# Construct and simulate circuit using the split-operator method
circuit = cirq.Circuit.from_ops(
    openfermioncirq.bogoliubov_transform(
        qubits, transformation_matrix, initial_state=initial_state),
    openfermioncirq.simulate_trotter(
        qubits, hamiltonian, time, n_steps, order,
        algorithm=openfermioncirq.trotter.SPLIT_OPERATOR)
)
result = simulator.simulate(
    circuit,
    initial_state=initial_state,
    qubit_order=qubits)
final_state = result.final_state

print('Energy of state obtained with split-operator method: {}'.format(
    openfermion.expectation(hamiltonian_sparse, final_state).real))

Energy of state obtained with swap network method: 19.426866262586053
Energy of state obtained with split-operator method: 19.424703863074306
